In [1]:
import os
import sys
import pandas as pd
import numpy as np
import pysam
from pybedtools import BedTool
from Bio import Seq

In [2]:
!date
!pwd
!conda env list | grep "*"

Fri 17 Dec 16:35:37 GMT 2021
/home/a/asmith/stopgap/asmith/capture_pipeline/CapCruncher
base                  *  /home/a/asmith/miniconda3/envs/cc


In [3]:
#!cp /tmp/pytest-of-asmith/pytest-183/datacurrent/capcruncher_preprocessing/restriction_enzyme_map/genome.digest.bed.gz .

# Set-up

In [4]:
viewpoint = BedTool("tests/data/data_for_pipeline_run/mm9_capture_oligos_Slc25A37.bed")[0]

In [5]:
restriction_fragments = BedTool("genome.digest.bed.gz")

In [6]:
df_restriction_fragments = restriction_fragments.to_dataframe()

In [7]:
vp_fragment = df_restriction_fragments.query("(start > @viewpoint.start) and (end < @viewpoint.end)")

In [8]:
fasta = "tests/data/data_for_pipeline_run/chr14.fa"

### Get adjacent fragments

In [9]:
adjacent_fragments = np.concatenate([(vp_fragment["name"] + adj).values for adj in [1, -1]])

# Filters to test:

- remove_unmapped_slices
- remove_orphan_slices
- remove_multi_capture_fragments
- remove_excluded_slices
- remove_blacklisted_slices
- remove_non_reporter_fragments
- remove_multicapture_reporters
- remove_slices_without_re_frag_assigned
- remove_duplicate_re_frags
- remove_duplicate_slices
- remove_duplicate_slices_pe
- remove_non_reporter_fragments


All reads will be 150bp/150bp with 25 bp shared between the capture and the viewpoint

### Viewpoint base slice

In [10]:
vp_base_start = (vp_fragment["end"] - 154).values[0]
vp_base_end = (vp_base_start + 150)

In [11]:
vp_base_end - vp_base_start

150

### Unmapped slices (gibberish)

In [12]:
def get_viewpoint_overlap(fasta, chrom, viewpoint_end, viewpoint_overlap, add_restriction_site=True):
    start = viewpoint_end - viewpoint_overlap
    end = start + viewpoint_overlap
    seq = BedTool().seq((chrom, start, end), fasta)
    
    if add_restriction_site:
        seq = "".join([seq, "GATC"])
        
    return seq

In [13]:
def get_unmapped_read(fasta, chrom, viewpoint_end, viewpoint_overlap, length):
    unmapped_vp_seq = get_viewpoint_overlap(fasta, chrom, viewpoint_end, viewpoint_overlap, add_restriction_site=True)
    unmapped_slice = "".join(np.random.choice(["A", "G", "T", "C"], (length - viewpoint_overlap - 4)))
    return "".join([unmapped_vp_seq, unmapped_slice])
    

In [14]:
unmaped_sequences = [get_unmapped_read(fasta, "chr14", vp_base_end, 25, 150) for _ in range(10)]
unmaped_sequences

['CAACCAGCTTTAGCATTCCGAAGGCGATCTGCGGGCCGAGCCCCAATAATGGAAATGACGTGATCTGCGCTATTTGCGTTTGTTCGTGCCAATACTTCGGCGTGTCTGGACGATAGCGCCCGAGAACGGCAGCTGTACCATATAAGTGCG',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCCTATAACTGTGAGATCAATGCCGCCTCCCGATCCCGTCCTGGCAGCGATCGAGGCGCTCGAGGGAATCGTTCTCGAGGCCTCCGTCAGGTGTGCTGCGAGGCTTTATTCAGAATCGGGTAA',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCCTGACTACGACGCGATGGGTTGATTTTAATTTAGCAAGAGTTCAAAGAATAGCAGAGTGTCCGTAAGTCGTGCCTGCCCGTCCGATACTTACCCTATAGTTCCCTATTCCATGTTTGTCTG',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCAAACCATCCGTCAGCCGGTACCTGTTGGGTACTCTGAACTAAGTTACCATAAAACCGGTTTAATTTGAAGTGTTTGGGCACCTCACATACCTGAGTGGGTATGAGAATCGGTGTGGACCCA',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCGGCAGCACGCGTCAACTTGAGATAGTATCAACTCAGCTCGTGACTGTGCATGCTCGGTTATGACTGAGCCATGAAACTCGACAGACATGAATAATTTGCTACCATGGAGGGGCCTGTGGTC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCGAAAGGCAGGGATAGGCACCTGCCGGGTAGTGCCGTACCGGCGATGTTCAGCTAATCCACGACACTATCCAGCCTACGAGTAAGTTTTCTTTCATCGTTAATCAAATTGGGCCATTCTGCT',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCACTTGTCCATTCGTATCACTAGAATTTATGTAACTGTGG

### Orphan slices  (no viewpoint, anywhere in the genome)

In [15]:
bt = BedTool()

In [16]:
fasta_extent = [len(chrom.sequence) for chrom in pysam.FastxFile(fasta)][0]

In [17]:
orphan_starts = np.random.randint(0, high=fasta_extent, size=(10,)) 
orphan_ends = orphan_starts + 150
orphan_sequences = [bt.seq(("chr14", start, end), fasta) for (start, end) in zip(orphan_starts, orphan_ends)]
orphan_sequences_with_cutsite = ["".join([seq[:len(seq)//2], "GATC", seq[len(seq)//2:-4]]) for seq in orphan_sequences]

In [18]:
orphan_sequences_with_cutsite

['ACTGAACAAGAATTACATTGTATACAAACCACATTTTCTTTTCTTTCAAAAAATCGTTTTTgccagacgtggtagGATCtacacccctttaatcccagcacttgggaggcagaggcaggcagatttctgagttcaaggccagcctggtct',
 'GTCTTACACCAAAGGCATTTCCAATCTTGCAGGCGAAGCACACTGCCACAGGTTTTGTCACAGACTGCACACTTGGATCGCACTCACAGGCAAGTTGCCTTCGAGCCACTGGTGAGGCATGGCAACCTGCGGACAGGAAGAAGTAATGGT',
 'AGGGACTTGGAGGATAATAGACCTGGAAAATATCTTGGAGGAGGTAAATCTTGTATGCATTTTGAGTCCTAAAAGGATCTGCTTTTAGCTTTGGAAGTGGGGACTAATTAGAAATACTTGCCTTTATAATTCAAATCCTCATTTAATATT',
 'TATGATGTGTATAGtattatagtgtagtataccaactataatatactatactataccactatagtattactatatGATCagtattatagtatacaatacCACTATAGTATTAATGGATTATGAAGTCTGATTTAACCCTGACCAATGCTA',
 'AAGTGCTAACTGTAAAGTTAGAAGATATTTTGGGGATGTAGGGAAGGATTCTGGGCACTGTGTCCTTGGGAATTCGATCTGAGAGGAACAGTGGAGGAAGGGCTGTCTAACGCTCATGGAAGTGACTGTCCAATATTTTAGGATTCAATG',
 'CCGTATAAAATTCTAATTGCCGTTAAATGTTTTCTTTAAACTAAAAATCTTTATTAGAAGAACAGAAggctgaggGATCagatggttcagtgggtaaggcatgggctgtgtgagcacgaggaattgagttcagatcttcaggacccacat',
 'taaagctgacatattgcttagaggaattgagacaggctgttaagtccagttatactataccttgttat

### Multicapture (N/A for now)

### Excluded

In [19]:
def get_sequences_from_fragment(fragment_coords: pd.Series, fasta: str, viewpoint_end: int, n_sequences=10):
    starts = np.random.randint(low=fragment_coords["start"], high=fragment_coords["end"], size=(n_sequences))
    ends = starts + (150 - 25 - 4)
    chrom = fragment_coords["chrom"]
    sequences = ["".join([get_viewpoint_overlap(fasta, chrom, viewpoint_end, 25), bt.seq((chrom, start, end), fasta)]) for start, end in zip(starts, ends)]
    return sequences

In [20]:
adj_fragment_coords = df_restriction_fragments.loc[adjacent_fragments[0]]

In [21]:
# adj_starts = np.random.randint(low=adj_fragment_coords["start"], high=adj_fragment_coords["end"], size=(10))
# adj_ends = adj_starts + (150 - 25 - 4)
# excluded_sequences = ["".join([get_viewpoint_overlap(fasta, "chr14", vp_base_end, 25), bt.seq(("chr14", start, end), fasta)]) for start, end in zip(adj_starts, adj_ends)]

In [22]:
excluded_sequences = get_sequences_from_fragment(adj_fragment_coords, fasta, vp_base_end)

### Blacklisted (N/A for now)

### Non reporter (Just a viewpoint slice)

Can just use the gibberish slices for this as they will just have one slice

### Duplicate restriction fragments

In [23]:
same_re_fragment_coords = df_restriction_fragments.loc[adjacent_fragments[0] + 100]

In [24]:
same_re_fragment_coords

chrom       chr14
start    69945565
end      69945713
name       169737
Name: 169737, dtype: object

In [25]:
same_starts = np.random.randint(low=same_re_fragment_coords["start"], high=same_re_fragment_coords["end"], size=(10))
same_mids = same_starts + ((150 - 25 - 8) // 2)
same_ends = same_starts + 150 - 8 - 25

In [26]:
same_fragment_sequences = ["".join([get_viewpoint_overlap(fasta, "chr14", vp_base_end, 25), 
          bt.seq(("chr14", start, mid), fasta), 
          "GATC", 
          bt.seq(("chr14", mid, end), fasta).replace("GATC", "GATT")]) 
 for start, mid, end in zip(same_starts, same_mids, same_ends)]

### Duplicates (Flashed)

Will make duplicates for this fragment
chr14:69922733-69923656


Need to edit one of the bases at random in the slice to stop the duplicate filter from removing this


In [27]:
def mutate_basepair(seq, start, end):
    mutation_index = np.random.randint(start, end)
    replacement = np.random.choice(["G", "A", "T", "C"])
    return "".join([seq[:mutation_index], replacement, seq[mutation_index+1:]])

In [28]:
duplicate_coords_sequences = ["".join([get_viewpoint_overlap(fasta, "chr14", vp_base_end, 25),  bt.seq(("chr14", 69922733, 69922733 + 150 - 25 - 4), fasta)]) for _ in range(10)]
duplicate_coords_sequences_random_mutation = [mutate_basepair(seq, 30, 120) for seq in duplicate_coords_sequences]

In [29]:
duplicate_coords_sequences_random_mutation

['CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTAAGGCTCTGGGCAGAAAAGGAACAAGAACATAGTCGGGTTCAGAAGTTTTATCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTAAGGCTCTGGGCAGAAAAGGAACAAGAACATAGCCGAGTTCAGAAGTTTTATCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTAAGGCTCTGGGCAGAAAAGGAACAAGAACATAGCCGGGTTCAGAAGTTTTAGCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTATGGCTCTGGGCAGAAAAGGAACAAGAACATAGCCGGGTTCAGAAGTTTTATCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTAAGGCTCTGGGCAGAAAAGGAACAAGAACATAGCCGGGTTCAGAAGTTTTATCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCCCTAAGGCTCTGGGCAGAAAAGGAACAAGAACATAGCCGGGTTCAGAAGTTTAATCTTTAAGCTTATTTCTGGCTGGGAGTACCCAGCACAGCCTATCATAGTTTTTTAAAACAGC',
 'CAACCAGCTTTAGCATTCCGAAGGCGATCTCCTCTCTAAGGCTCTGGGCAGAAAAGGAACAAGAACAT

### True reporters (Flashed)

Will use this fragment:
chr14:69878304-69878786

In [30]:
reporter_coords = {"chrom":"chr14", "start":69878304, "end":69878786}

In [31]:
reporter_sequences = get_sequences_from_fragment(reporter_coords, fasta, vp_base_end, n_sequences=100)

## True reporters (PE)

In [32]:
starts = np.random.randint(reporter_coords["start"],reporter_coords["end"], size=(100,))
ends = starts + 146
reporter_sequences_pe = [bt.seq((reporter_coords["chrom"], start, end), fasta) for start, end in zip(starts, ends)]

# Generate fastq file using these data

Highest quality == "~"

In [33]:
viewpoint_sequence = bt.seq(("chr14", vp_base_end-154, vp_base_end+4), fasta)
viewpoint_sequence

'CTAGAGGGTGGTGACCATCTGCAAGGGACAGTCAGGTCCCTGGGGAGAGTCAGCAGAAGGGATATGCTCAGAGCTGCAAGAGCAGGAGAAGGGGCTGTCCCCAGAATCGGTAGTTTTGCAGACACTAATCAACCAGCTTTAGCATTCCGAAGGCTGGG'

In [34]:
sequences_for_fastq = {"unmapped": unmaped_sequences, 
                       "orphan": orphan_sequences_with_cutsite, 
                       "excluded": excluded_sequences, 
                       "duplicate_rf": same_fragment_sequences,
                       "duplicate_coords": duplicate_coords_sequences_random_mutation,
                       "reporters_flashed": reporter_sequences,
                       "reporters_pe": reporter_sequences_pe,}

In [35]:
SAMPLE_NAME = "SAMPLE-B"

In [36]:
n_reporters = {k: len(v) for k, v in sequences_for_fastq.items()}
ser = pd.Series(n_reporters)
ser.to_frame("n_reads").rename_axis(index="category").reset_index().to_csv(f"tests/data/data_for_pipeline_run/{SAMPLE_NAME}", index=False) 

In [37]:
with open("SAMPLE-B_REP2_1.fastq", "w") as writer:
    for category, sequences in sequences_for_fastq.items():
        for ii, sequence in enumerate(sequences):
            if not category == "orphan":
                record = pysam.FastxRecord(name=f"{category}_{ii}", sequence=viewpoint_sequence, quality="".join(["~" for _ in range(len(viewpoint_sequence))]))
                writer.write(str(record) + "\n")
            else:
                # Just use the orphan sequence twice in reverse
                record = pysam.FastxRecord(name=f"{category}_{ii}", sequence=sequence[::-1], quality="".join(["~" for _ in range(len(sequence))]))
                writer.write(str(record) + "\n")   

### Might need to reverse the sequence to convince flash

In [38]:
with open("SAMPLE-B_REP2_2.fastq", "w") as writer:
    for category, sequences in sequences_for_fastq.items():
        for ii, sequence in enumerate(sequences):
            record = pysam.FastxRecord(name=f"{category}_{ii}", sequence=str(Seq.Seq(sequence).reverse_complement()), quality="".join(["~" for _ in range(len(sequence))]))
            writer.write(str(record) + "\n")

In [39]:
!mv SAMPLE-*_REP*.fastq tests/data/data_for_pipeline_run/

In [40]:
!ls tests/data/data_for_pipeline_run/ -l

total 164488
drwxr-sr-x 2 asmith milne_groupgrp       116 Dec 16 16:43 chr14_bowtie2_indicies
-rw-r--r-- 1 asmith milne_groupgrp 127698769 Jul 20 15:49 chr14.fa
-rw-r--r-- 1 asmith milne_groupgrp        24 Jul 20 15:49 chr14.fa.fai
-rw-r--r-- 1 asmith milne_groupgrp  39935628 Dec 17 12:43 chr14.fa.gz
-rw-r--r-- 1 asmith milne_groupgrp       149 Sep 13 16:59 design_matrix.tsv
-rw-r--r-- 1 asmith milne_groupgrp        33 Dec 16 14:00 mm9_capture_oligos_Slc25A37.bed
-rw-r--r-- 1 asmith milne_groupgrp    157998 Sep 13 16:59 mm9_chr14_genes.bed
-rw-rw-r-- 1 asmith milne_groupgrp     60089 Dec 17 16:25 mm9_chr14_genes.bed.bgz
-rw-rw-r-- 1 asmith milne_groupgrp      3380 Dec 17 16:13 mm9_chr14_genes.bed.bgz.tbi
drwxr-sr-x 2 asmith milne_groupgrp      4096 Dec 17 12:39 old
-rw-r--r-- 1 asmith milne_groupgrp        61 Sep 13 16:59 plot_coords.bed
-rw-r--r-- 1 asmith milne_groupgrp        33 Nov 25 17:28 regions_for_norm.bed
-rw-r--r-- 1 asmith milne_groupgrp       124 Dec 17 16:33 SAMPLE-A
-rw-

In [41]:
!flash tests/data/data_for_pipeline_run/SAMPLE-B_REP1_1.fastq tests/data/data_for_pipeline_run/SAMPLE-B_REP1_2.fastq

[FLASH] Starting FLASH v1.2.11
[FLASH] Fast Length Adjustment of SHort reads
[FLASH]  
[FLASH] Input files:
[FLASH]     tests/data/data_for_pipeline_run/SAMPLE-B_REP1_1.fastq
[FLASH]     tests/data/data_for_pipeline_run/SAMPLE-B_REP1_2.fastq
[FLASH]  
[FLASH] Output files:
[FLASH]     ./out.extendedFrags.fastq
[FLASH]     ./out.notCombined_1.fastq
[FLASH]     ./out.notCombined_2.fastq
[FLASH]     ./out.hist
[FLASH]     ./out.histogram
[FLASH]  
[FLASH] Parameters:
[FLASH]     Min overlap:           10
[FLASH]     Max overlap:           65
[FLASH]     Max mismatch density:  0.250000
[FLASH]     Allow "outie" pairs:   false
[FLASH]     Cap mismatch quals:    false
[FLASH]     Combiner threads:      40
[FLASH]     Input format:          FASTQ, phred_offset=33
[FLASH]     Output format:         FASTQ, phred_offset=33
[FLASH]  
[FLASH] Starting reader and writer threads
[FLASH] Starting 40 combiner threads
[FLASH] Processed 250 read pairs
[FLASH]  
[FLASH] Read combination statistics:
[FLAS